In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, auc,roc_curve
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import ttest_ind
import time
from datetime import date
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
from sklearn import svm
import scipy

In [49]:
df = pd.read_csv('train.csv', parse_dates=['last_review', 'host_since'])
df1 = df.copy()

In [50]:
features_mean = list(df.columns[1:24])
df.columns.values.tolist()
feat = [0,4,7,8,18,10,5,9,11,12,15]
sort= [5,9]
nah = [10,11,12,15]
feature_list = []
for i in feat:
    feature_list += [features_mean[i]]

In [51]:
def encodeval(dataframe,features_mean,col):
    dataF = dataframe.copy()
    le = preprocessing.LabelEncoder()
    le.fit(dataF[features_mean[col]])
    dataF[features_mean[col]] = le.transform(dataF[features_mean[col]])
    return dataF

df1['last_review'] = df['last_review'].dt.month
df['host_since'] = pd.to_datetime(df['host_since']).dt.date
df1['host_since'] = -(df['host_since']- date(2020, 10, 5)).dt.days
df1 = encodeval(df1,features_mean,0)
df1 = encodeval(df1,features_mean,1)
df1 = encodeval(df1,features_mean,9)
df1 = encodeval(df1,features_mean,13)
df1 = encodeval(df1,features_mean,18)
df1 = encodeval(df1,features_mean,19)
df1 = encodeval(df1,features_mean,20)
df1 = encodeval(df1,features_mean,21)
df1 = encodeval(df1,features_mean,22)
features_mean.pop(19)

'is_business_travel_ready'

In [52]:
X = df1.loc[:,features_mean]
X = (X.iloc[:,0:] - X.iloc[:,0:].mean()) / X.iloc[:,0:].std() #standardise
y = df1.loc[:, 'price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

In [54]:
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_val, y_val, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

[18:07:05] 6486x22 matrix with 142692 entries loaded from dtrain.svm
[18:07:05] 3195x22 matrix with 70290 entries loaded from dtest.svm


In [55]:
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5}  # the number of classes that exist in this datset
num_round = 100  # the number of training iterations

In [56]:
bst = xgb.train(param, dtrain, num_round)

In [57]:
bst.dump_model('dump.raw.txt')

In [58]:
preds = bst.predict(dtest)

In [59]:
best_preds = np.asarray([np.argmax(line) for line in preds])

In [60]:
print(precision_score(y_val, best_preds, average='macro'))

0.540455695326385


In [61]:
print(accuracy_score(y_val, best_preds))

0.5311424100156494


In [62]:
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [63]:
accuracy_score(y_val, y_pred)

0.49076682316118936

In [64]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



# creating tensor from targets_df 
X_traintorch = torch.tensor(X_train.values)
y_traintorch = torch.tensor(y_train.values)

X_valtorch = torch.tensor(X_val.values)
y_valtorch = torch.tensor(y_val.values)



In [65]:
class SimpleNet(nn.Module):
    def __init__(self, num_features, num_class):
        super(SimpleNet, self).__init__()
        self.layer_1 = nn.Linear(num_features, num_features//2)
        self.layer_out = nn.Linear(num_features//2, num_class) 
        
        
        #self.lr = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(num_features//2)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.bn(x)
        #x = self.lr(x)        
        x = self.layer_out(x)
        
        return x

In [68]:
model = SimpleNet(22,5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1)

In [69]:
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(X_traintorch.float())
    loss = criterion(outputs, y_traintorch)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs.data, 1)
    total += y_traintorch.size(0)
    correct += (predicted == y_traintorch).sum().item()
    
    # print statistics
    running_loss += loss.item()
    print('[%d] loss: %.3f acc: %.3f' %
              (epoch + 1, running_loss / 2000, (100 * correct / total)))

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    outputs = model(X_valtorch.float())
    _, predicted = torch.max(outputs.data, 1)
    total += y_valtorch.size(0)
    correct += (predicted == y_valtorch).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

[1] loss: 0.001 acc: 13.537
[2] loss: 0.002 acc: 22.387
[3] loss: 0.004 acc: 25.254
[4] loss: 0.003 acc: 35.338
[5] loss: 0.003 acc: 38.915
[6] loss: 0.003 acc: 34.413
[7] loss: 0.003 acc: 25.979
[8] loss: 0.002 acc: 37.142
[9] loss: 0.002 acc: 33.935
[10] loss: 0.001 acc: 29.695
[11] loss: 0.001 acc: 37.882
[12] loss: 0.001 acc: 39.578
[13] loss: 0.001 acc: 31.221
[14] loss: 0.001 acc: 29.988
[15] loss: 0.001 acc: 36.247
[16] loss: 0.001 acc: 38.452
[17] loss: 0.001 acc: 34.289
[18] loss: 0.001 acc: 38.961
[19] loss: 0.001 acc: 44.018
[20] loss: 0.001 acc: 40.965
[21] loss: 0.001 acc: 35.353
[22] loss: 0.001 acc: 42.291
[23] loss: 0.001 acc: 41.906
[24] loss: 0.001 acc: 40.056
[25] loss: 0.001 acc: 46.469
[26] loss: 0.001 acc: 44.619
[27] loss: 0.001 acc: 44.172
[28] loss: 0.001 acc: 46.747
[29] loss: 0.001 acc: 42.538
[30] loss: 0.001 acc: 42.985
[31] loss: 0.001 acc: 42.754
[32] loss: 0.001 acc: 44.989
[33] loss: 0.001 acc: 41.751
[34] loss: 0.001 acc: 44.527
[35] loss: 0.001 acc: 4

[283] loss: 0.001 acc: 49.522
[284] loss: 0.001 acc: 49.537
[285] loss: 0.001 acc: 49.553
[286] loss: 0.001 acc: 49.537
[287] loss: 0.001 acc: 49.507
[288] loss: 0.001 acc: 49.507
[289] loss: 0.001 acc: 49.507
[290] loss: 0.001 acc: 49.522
[291] loss: 0.001 acc: 49.507
[292] loss: 0.001 acc: 49.522
[293] loss: 0.001 acc: 49.537
[294] loss: 0.001 acc: 49.537
[295] loss: 0.001 acc: 49.522
[296] loss: 0.001 acc: 49.522
[297] loss: 0.001 acc: 49.553
[298] loss: 0.001 acc: 49.537
[299] loss: 0.001 acc: 49.522
[300] loss: 0.001 acc: 49.522
[301] loss: 0.001 acc: 49.507
[302] loss: 0.001 acc: 49.491
[303] loss: 0.001 acc: 49.507
[304] loss: 0.001 acc: 49.507
[305] loss: 0.001 acc: 49.507
[306] loss: 0.001 acc: 49.491
[307] loss: 0.001 acc: 49.491
[308] loss: 0.001 acc: 49.491
[309] loss: 0.001 acc: 49.460
[310] loss: 0.001 acc: 49.460
[311] loss: 0.001 acc: 49.476
[312] loss: 0.001 acc: 49.476
[313] loss: 0.001 acc: 49.460
[314] loss: 0.001 acc: 49.460
[315] loss: 0.001 acc: 49.476
[316] loss

[574] loss: 0.001 acc: 49.584
[575] loss: 0.001 acc: 49.584
[576] loss: 0.001 acc: 49.584
[577] loss: 0.001 acc: 49.584
[578] loss: 0.001 acc: 49.584
[579] loss: 0.001 acc: 49.584
[580] loss: 0.001 acc: 49.584
[581] loss: 0.001 acc: 49.584
[582] loss: 0.001 acc: 49.584
[583] loss: 0.001 acc: 49.584
[584] loss: 0.001 acc: 49.584
[585] loss: 0.001 acc: 49.584
[586] loss: 0.001 acc: 49.584
[587] loss: 0.001 acc: 49.584
[588] loss: 0.001 acc: 49.584
[589] loss: 0.001 acc: 49.584
[590] loss: 0.001 acc: 49.584
[591] loss: 0.001 acc: 49.584
[592] loss: 0.001 acc: 49.584
[593] loss: 0.001 acc: 49.584
[594] loss: 0.001 acc: 49.584
[595] loss: 0.001 acc: 49.584
[596] loss: 0.001 acc: 49.584
[597] loss: 0.001 acc: 49.584
[598] loss: 0.001 acc: 49.584
[599] loss: 0.001 acc: 49.584
[600] loss: 0.001 acc: 49.584
[601] loss: 0.001 acc: 49.584
[602] loss: 0.001 acc: 49.584
[603] loss: 0.001 acc: 49.584
[604] loss: 0.001 acc: 49.584
[605] loss: 0.001 acc: 49.584
[606] loss: 0.001 acc: 49.584
[607] loss

[853] loss: 0.001 acc: 49.645
[854] loss: 0.001 acc: 49.537
[855] loss: 0.001 acc: 49.630
[856] loss: 0.001 acc: 49.568
[857] loss: 0.001 acc: 49.584
[858] loss: 0.001 acc: 49.584
[859] loss: 0.001 acc: 49.553
[860] loss: 0.001 acc: 49.645
[861] loss: 0.001 acc: 49.599
[862] loss: 0.001 acc: 49.630
[863] loss: 0.001 acc: 49.599
[864] loss: 0.001 acc: 49.630
[865] loss: 0.001 acc: 49.507
[866] loss: 0.001 acc: 49.584
[867] loss: 0.001 acc: 49.198
[868] loss: 0.001 acc: 49.306
[869] loss: 0.001 acc: 48.181
[870] loss: 0.001 acc: 47.872
[871] loss: 0.001 acc: 47.872
[872] loss: 0.001 acc: 47.549
[873] loss: 0.001 acc: 48.952
[874] loss: 0.001 acc: 49.553
[875] loss: 0.001 acc: 48.689
[876] loss: 0.001 acc: 48.551
[877] loss: 0.001 acc: 49.615
[878] loss: 0.001 acc: 49.337
[879] loss: 0.001 acc: 49.337
[880] loss: 0.001 acc: 49.707
[881] loss: 0.001 acc: 49.322
[882] loss: 0.001 acc: 49.692
[883] loss: 0.001 acc: 49.553
[884] loss: 0.001 acc: 49.661
[885] loss: 0.001 acc: 49.245
[886] loss

In [70]:
XGB = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping=5, silence=False)
XGB.fit(X_train, y_train)
y_pred = XGB.predict(X_val)

print(accuracy_score(y_val, np.around(y_pred)))

[18:08:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.49827856025039124


In [71]:
np.around(y_pred)

array([2., 1., 3., ..., 1., 1., 2.], dtype=float32)

In [72]:
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LinearRegression
reg=LogisticRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_val)

print(accuracy_score(y_val,y_pred ))

0.4682316118935837


In [73]:
from sklearn.tree import DecisionTreeRegressor
DTree=DecisionTreeRegressor()
DTree.fit(X_train,y_train)
y_pred=DTree.predict(X_val)

print(accuracy_score(y_val,y_pred ))

0.43380281690140843


In [74]:
import lightgbm as lgb

In [75]:
train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':150, 'max_depth':7,'learning_rate':.05,'max_bin':200}

val_data=lgb.Dataset(X_val,label=y_val)
clf = lgb.train(param,train_data,100)
y_pred=clf.predict(X_val)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000704 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1467
[LightGBM] [Info] Number of data points in the train set: 6486, number of used features: 22
[LightGBM] [Info] Start training from score 2.491520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [76]:
print(accuracy_score(y_val,np.around(y_pred)))

0.49827856025039124


In [106]:
# Define initial best params and MAE
params = {
    'n_estimators': 50,
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5} 

dvals = xgb.DMatrix(X, label=y)

n_estimators = range(50, 400, 25)

min_mae = float("Inf")
best_params = None
gridsearch_params = [
    (max_depth, min_child_weight,n_estimators)
    for max_depth in range(3,20)
    for min_child_weight in range(2,15)
    for n_estimators in range(50,400,25)
]
for max_depth, min_child_weight, n_estimators in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}, n_estimators={}".format(
                             max_depth,
                             min_child_weight, n_estimators))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    params['n_estimators'] = n_estimators
    # Run CV
    cv_results = xgb.cv(
        params,
        dvals,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {},, {}, MAE: {}".format(best_params[0], best_params[1],best_params[2], min_mae))

CV with max_depth=3, min_child_weight=2, n_estimators=50


XGBoostError: [00:12:04] C:/Jenkins/workspace/xgboost-win64_release_0.90/src/metric/elementwise_metric.cu:326: Check failed: preds.Size() == info.labels_.Size() (38720 vs. 7744) : label and prediction size not match, hint: use merror or mlogloss for multi-class classification

In [92]:
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5}  # the number of classes that exist in this datset
num_round = 100  # the number of training iterations
params['max_depth'] = 4
params['min_child_weight'] = 3

In [93]:
bst = xgb.train(param, dtrain, num_round)

In [94]:
bst.dump_model('dump.raw.txt')

In [95]:
preds = bst.predict(dtest)

In [96]:
best_preds = np.asarray([np.argmax(line) for line in preds])

In [97]:
print(precision_score(y_val, best_preds, average='macro'))

0.540455695326385


In [98]:
print(accuracy_score(y_val, best_preds))

0.5311424100156494


In [89]:
y_val

6933    1
7012    1
5709    3
4453    2
7395    3
       ..
9073    2
3150    3
5227    1
5573    1
3497    3
Name: price, Length: 3195, dtype: int64

In [103]:
gridsearch_params = [
    (max_depth, min_child_weight, n_estimators)
    for max_depth in range(3,20)
    for min_child_weight in range(2,15)
    for n_estimators in range(50, 400, 25)
]

In [104]:
gridsearch_params

[(3, 2, 50),
 (3, 2, 75),
 (3, 2, 100),
 (3, 2, 125),
 (3, 2, 150),
 (3, 2, 175),
 (3, 2, 200),
 (3, 2, 225),
 (3, 2, 250),
 (3, 2, 275),
 (3, 2, 300),
 (3, 2, 325),
 (3, 2, 350),
 (3, 2, 375),
 (3, 3, 50),
 (3, 3, 75),
 (3, 3, 100),
 (3, 3, 125),
 (3, 3, 150),
 (3, 3, 175),
 (3, 3, 200),
 (3, 3, 225),
 (3, 3, 250),
 (3, 3, 275),
 (3, 3, 300),
 (3, 3, 325),
 (3, 3, 350),
 (3, 3, 375),
 (3, 4, 50),
 (3, 4, 75),
 (3, 4, 100),
 (3, 4, 125),
 (3, 4, 150),
 (3, 4, 175),
 (3, 4, 200),
 (3, 4, 225),
 (3, 4, 250),
 (3, 4, 275),
 (3, 4, 300),
 (3, 4, 325),
 (3, 4, 350),
 (3, 4, 375),
 (3, 5, 50),
 (3, 5, 75),
 (3, 5, 100),
 (3, 5, 125),
 (3, 5, 150),
 (3, 5, 175),
 (3, 5, 200),
 (3, 5, 225),
 (3, 5, 250),
 (3, 5, 275),
 (3, 5, 300),
 (3, 5, 325),
 (3, 5, 350),
 (3, 5, 375),
 (3, 6, 50),
 (3, 6, 75),
 (3, 6, 100),
 (3, 6, 125),
 (3, 6, 150),
 (3, 6, 175),
 (3, 6, 200),
 (3, 6, 225),
 (3, 6, 250),
 (3, 6, 275),
 (3, 6, 300),
 (3, 6, 325),
 (3, 6, 350),
 (3, 6, 375),
 (3, 7, 50),
 (3, 7, 75),
 (3,